In [ ]:
!pip install -q findspark==1.3.0
!pip install py4j==0.10.9

# For plotting
!pip install folium
!pip install plotly

!pip install psycopg2
!pip install pandas
!pip install tweepy

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"] = ""

In [2]:
import findspark
findspark.add_packages(["org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"])
findspark.add_jars(["/content/spark-3.0.1-bin-hadoop2.7/jars/kafka-clients-2.0.0.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/lz4-java-1.4.1-jar","/content/spark-3.0.1-bin-hadoop2.7/jars/scala-library-2.11.12.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/slf4j-api-1.7.25.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/snappy-java-1.1.7.1.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/spark-sql-kafka-0-10_2.11-2.4.5.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/spark-tags_2.11-2.4.5.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/unused-1.0.0.jar"])
findspark.init("/content/spark-3.0.1-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4040") \
        .getOrCreate()

spark.version

'3.0.1'

In [3]:
spark

In [4]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [5]:
import psycopg2
import numpy as np
import pandas as pd
import time
import json
import tweepy
from tweepy import OAuthHandler
import requests
import os
import sys

In [6]:
def connect():
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "Welcome01",
                                      host = "104.154.145.52",
                                      port = "5432",
                                      database = "dataproject2")
       
        cursor = connection.cursor()
        return cursor
        
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
        

In [7]:
def algoritmo(text_person):
    tg0 = time.time()
    cursor = connect()
    
    playa = ["Valencia", "Barcelona", "Ibiza"]
    ciudad = ["Madrid", "Barcelona", "Valencia", "Sevilla"]
    naturaleza = ["Oviedo", "Bilbao"]
    fiesta = ["Ibiza", "Madrid", "Barcelona"]
    
    scoring = {"Madrid": 0, "Barcelona": 0, "Ibiza": 0, "Valencia": 0, "Sevilla": 0, "Oviedo": 0, "Bilbao": 0}
    
    json_person = json.loads(text_person)
    tweet_text = json_person['full_text']
    tweet_person_id = json_person['id']
    
    split_text = tweet_text.split(" ")
    try:
        name = split_text[3] + " " + split_text[4][:-1]
        salary = split_text[8][:-1]
        members = split_text[19]
        beach = split_text[25][-3:-2]
        city = split_text[26][-3:-2]
        nature = split_text[27][-3:-2]
        party = split_text[28][-2:-1]

        t0 = time.time()
        
        for k in scoring:
            try:
                playa.index(k)
                scoring[k] += int(beach)
            except (Exception) as error :
                print("Not present", error)

            try:
                ciudad.index(k)
                scoring[k] += int(city)
            except (Exception) as error :
                print("Not present", error)

            try:
                naturaleza.index(k)
                scoring[k] += int(nature)
            except (Exception) as error :
                print("Not present", error)

            try:
                fiesta.index(k)
                scoring[k] += int(party)
            except (Exception) as error :
                print("Not present", error)

        t1 = time.time()
        print("Tiempo de scoring: "+ str(t1-t0))
        
        t0 = time.time()
        cursor.execute("SELECT * FROM casas WHERE cost <= " + str((int(salary)/12)*0.2) + " and (rooms = "
                       + str(members) + " or rooms = " + str(int(members)+1) + ") and c_counter <=4;")
        record = cursor.fetchall()

        cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'casas'")
        columns_name = cursor.fetchall()
        
        t1 = time.time()
        print("Tiempo de consulta a base de datos: "+ str(t1-t0))

        """Convert array of arrays to single array"""

        array_columns_name = np.array(columns_name)
        array_columns_name = np.concatenate( array_columns_name, axis=0 )

        print(array_columns_name)

        """Transform result of query to a pandas dataframe"""

        df = pd.DataFrame(record, columns=array_columns_name)

        #casas_coste = df[df.cost <= ((int(salary)/12)*0.2)]

        #casas_hab = casas_coste.loc[(casas_coste.rooms == members) | ((casas_coste.rooms.astype(int)) == int(members)+1)]

        #casas_libres = casas_hab.loc[casas_hab.c_counter <= 4]
        mejor_casa = df
        mejor_casa['score'] = mejor_casa['city_name'].map(scoring)
        resultado = mejor_casa[mejor_casa.score == max(mejor_casa.score)]

        consumer_key = "ZHwb6JawdsgTUZreb4yZeb2tX"
        consumer_secret = "2UoU6lEpNeTPM847paK6X6z6BVoUENhPq87rIIWcOycnNhGtpd"
        access_token = "1346037018132930560-G3afFHkhsPiYlo1yFO3X4mTmdJhTOY"
        access_token_secret = "jLjtkTK9VQVitacVhLj8QtSdb0mkOto4J9PDDSWR9x9Q0"

        images_url = {"Madrid": "https://www.enforex.com/images/fichas/madrid/ciudad-madrid-2.jpg", 
                  "Barcelona": "https://www.alsa.es/documents/21643679/21664598/Barcelona.jpg", 
                  "Ibiza": "https://www.iagua.es/sites/default/files/styles/thumbnail-700x700/public/1155x510-ibiza1.jpg?itok=0SafgsGX", 
                  "Valencia": "https://static.lasprovincias.es/www/multimedia/202009/30/media/cortadas/valencia-turismo-tarjeta-kfHG-U1203216296067wG-624x385@Las%20Provincias.jpg", 
                  "Sevilla": "https://elcorreoweb.es/binrepository/plaza-espana-sevilla_20333351_20200107162131.jpg", 
                  "Oviedo": "https://www.iberia.com/ibcomv3/content/landings/OVD.jpg", 
                  "Bilbao": "https://www.leonardo-hotels.es/octopus/Upload/images/Pages/bilbao-1920x580.jpg"}

        # Configuración de acceso con las credenciales
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)

        # Listos para hacer la conexión con el API
        api = tweepy.API(auth)

        t0 = time.time()
        contador = 0
        for _, i in resultado.iterrows():
            # Ofrecer como maximo 2 casas a una misma persona
            if contador >= 2:
                break;    

            tweet = "Hi! "+ name + ", your perfect house is placed in " + i.city_name + ", with the code: " + str(i.house_id)
            tweet += " https://twitter.com/dlpexercisepro1/status/" + str(i.tweet_id)

            t2 = time.time()
            # Download the image
            url = images_url.get(i.city_name)
            filename = '/tmp/temp.jpg'
            request = requests.get(url, stream=True)
            if request.status_code == 200:
                with open(filename, 'wb') as image:
                    for chunk in request:
                        image.write(chunk)
            else:
                return "Unable to download image"
            
            t3 = time.time()
            print("Tiempo de descarga de imagen: " +str(t3-t2))

            # Publish Tweet with image
            api.update_with_media(filename, status=tweet, in_reply_to_status_id = tweet_person_id, auto_populate_reply_metadata = True)

            os.remove(filename)   
            contador += 1

            # Increment house counter
            cursor.execute("UPDATE casas SET c_counter = c_counter + 1 WHERE tweet_id = " + str(i.tweet_id))
        t1 = time.time()
        print("Tiempo de publicación: "+ str(t1-t0))
        tg1 = time.time()
        print("Tiempo total: "+ str(tg1-tg0))
        return "publicado"
    except (Exception) as error :
        return "error"

In [8]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:29092") \
    .option("subscribe", "este") \
    .load()
    
#df.printSchema()  

schema = StructType(
    [
     StructField('created_at', StringType(), True),
     StructField('id', StringType(), True),
     StructField('full_text', StringType(), True),
     StructField('text', StringType(), True)
    ]
)

dataset = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*'))

#df1 = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
#        .select(col('key'), col('timestamp'), col('value'))

dataset.writeStream \
  .outputMode("update") \
  .format("memory") \
  .option("truncate", "false") \
  .queryName("tweets") \
  .start()

In [17]:
spark.sql("select * from tweets order by timestamp desc").show(truncate = False)

+----+-----------------------+------------------------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|key |timestamp              |created_at                    |id                 |full_text                                                                                                                                                                 |text                                                                                                                                        |
+----+-----------------------+------------------------------+-------------------+-----------------------------------------------------------------------------------------------------------------------------------

In [19]:
def spark_algoritmo(f, sp_id):
    return algoritmo(f.value.full_text)

In [20]:
from pyspark.sql.functions import *

spark.udf.register("udf_algoritmo", algoritmo) 
spark.udf.register("udf_spark_algoritmo", spark_algoritmo) 

dataset1 = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*'))

tweet_proc = dataset1.writeStream \
                .foreachBatch(spark_algoritmo) \
                .outputMode("update") \
                .format("memory") \
                .option("truncate", "false") \
                .queryName("output1") \
                .start()

In [22]:
spark.sql("select * from output1").show(truncate = False)

+----+----------------------+------------------------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|key |timestamp             |created_at                    |id                 |full_text                                                                                                                                                                 |text                                                                                                                                        |
+----+----------------------+------------------------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------